In [ ]:
 #This example uses Python 2.7 and the python-request library.

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'50',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '52b363ed-4aed-451c-9157-8cc986172de0',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd
pd.set_option('display.max_column', None)
pd.set_option('display.max_row', None)

In [ ]:
import datetime

df = pd.json_normalize(data['data'])
df['timestamp'] = datetime.datetime.now()
df

In [ ]:
def api_runner():
     #This example uses Python 2.7 and the python-request library.
    global df
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    import json
    import os
    from time import time
    from time import sleep
    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'20',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '52b363ed-4aed-451c-9157-8cc986172de0',
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    import pandas as pd
    import datetime
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = datetime.datetime.now()

    if not os.path.isfile(r"D:\nevinphilbert\Portfolio\Python Builder\Automated Crypto API Pull using Python\API Pull.csv"):
        df.to_csv(r"D:\nevinphilbert\Portfolio\Python Builder\Automated Crypto API Pull using Python\API Pull.csv", header='column_names')
    else:
        df.to_csv(r"D:\nevinphilbert\Portfolio\Python Builder\Automated Crypto API Pull using Python\API Pull.csv", mode='a', header=False)

In [ ]:
import os
from time import time
from time import sleep

for i in range(100):
    api_runner()
    print('API Runner completed')
    sleep(300) #sleep for 5 minutes
exit()

In [ ]:
df_new = pd.read_csv(r"D:\nevinphilbert\Portfolio\Python Builder\Automated Crypto API Pull using Python\API Pull.csv")
df_new

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df_new.columns.tolist()

In [ ]:
df2 = df_new.groupby('name', sort=False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()
df2

In [ ]:
dfs = df2.stack()
dfs

In [ ]:
type(dfs)

In [ ]:
df3 = dfs.to_frame(name='values')
df_fix = df3.reset_index()
df_fix

In [ ]:
type(df_fix)

In [ ]:
df1 = df_fix.rename(columns={'level_1': 'percent_change'})

In [ ]:
df1

In [ ]:
df1['percent_change'] = df1['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])

In [ ]:
pd.set_option('display.max_column', 30)
pd.set_option('display.max_row', None)
df1.head(50)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df1, kind='point', aspect=3)
plt.show()

In [ ]:
df_bitcoin_r = df_new[['name', 'quote.USD.price', 'timestamp']]
df_bitcoin_r['timestamp'] = pd.to_datetime(df_bitcoin_r['timestamp'])
df_bitcoin_r['timestamp'] = df_bitcoin_r['timestamp'].dt.strftime("%H:%M")
df_bitcoin = df_bitcoin_r.query("name == 'Bitcoin'")
#df_bitcoin['timestamp'] = df_bitcoin['timestamp'].dt.strftime("%Y-%m-%d %H:%M")
df_bitcoin

In [ ]:
sns.set_theme(style='darkgrid')
plt.figure(figsize=(30, 6))
sns.lineplot(x='timestamp', y='quote.USD.price', data=df_bitcoin)